# Processamento Digital de Imagens - Aula Prática 3

## 1. Preparação

### a) Edite a célula abaixo, preencha com seus dados pessoais e a execute novamente

<b>Aluno(a):</b>

- Matrícula: 98877
- Nome: gabriel macedo nunes pontes

### b) Observações e instruções gerais:

- Entregue <b>apenas</b> o notebook resultante do seu trabalho. Não há necessidade de se entregar qualquer outro arquivo.
- O trabalho deve ser desenvolvido <b>indivualmente</b>

### c) Execute a célula abaixo, que conterá as importações de módulos Python.

#### Você é livre para editar e adicionar outras importações a esta célula.

In [7]:
# Manipulação de imagens
from PIL import Image
from PIL import ImageOps

import numpy as np # Manipulação de arrays
import os # Acesso ao sistema de arquivos do SO

### d) Função auxiliar pronta. Execute a célula a seguir.

<p>Função <code>abre_imagem</code>:</p>
<ul>
    <li>A partir de um nome de arquivo, retorna dois objetos <code>PIL Image</code>, na versão RGB e em tons de cinza</li>
    <li>Em caso de erro ao encontrar arquivo, exibe mensagem de alerta em tela e retorna nulo (<code>None</code>)</li>
    <li><u>Parâmetro de entrada:</u> <code>arquivo</code>, <i>string</i> contendo o nome do arquivo de imagem</li>
    <li><u>Retorno:</u> objeto <code>PIL Image</code> na versão RGB da imagem</li>
</ul>

In [2]:
def abre_imagem(arquivo):
    if os.path.isfile(arquivo):
        img = Image.open(arquivo)
        if img.mode == 'L':
            img = img.convert('RGB')
        
        return img
    else:
        print('\n\n\nErro!!! Arquivo inexistente.\n\n\n')
        return None

## 2. Exercício: <i>Dithering</i>

<p><i>Dithering</i> (não há uma boa tradução para o termo) é uma técnica que consiste em gerar imagens binárias que, por meio de densidade de pixels pretos, gera um efeito que "engana" nosso sistema visual, trazendo-nos uma ilusão de uma imagem em tons de cinza, embora a mesma seja binária, isto é, só contenha pixels pretos e brancos.</p>

<p>Por densidade de pixels pretos, entenda-se a concentração de pixels pretos em uma mesma região da imagem.</p>

<p>Nesta aula prática, vamos implementar dois algoritmos diferentes de <i>dithering</i> o de Atkinson e o de Bayer.</p>

### 2.1 Algoritmo de Atkison

<p>É o mesmo algoritmo que está no slide $78$ do material sobre filtragem espacial:</p>

<img src="atkinson.png" width="80%">

<p><font color="red">Implemente o corpo da função</font> <code>atkinson</code>, na célula a seguir para que os códigos das células de teste funcionem corretamente.</p>

<p>Resultado esperado para a imagem <code>blue_tang.jpg</code>:</p>

<img src="blue_tang_atk.png">

<p>Resultado esperado para a imagem <code>capivara.jpg</code>:</p>

<img src="capivara_atk.jpg" width="30%">

<p>Resultado esperado para a imagem <code>praia.png</code>:</p>

<img src="praia_atk.png" width="25%">

<p><b>Obs.:</b> note que a imagem de entrada do algoritmo precisa estar em tons de cinza.</p>

In [27]:
def atkinson(img):
    img=ImageOps.grayscale(img)
    saida=np.asarray(img,dtype=int)
    for i in range(saida.shape[0]):
        for j in range(saida.shape[1]):
            salva=saida[i][j]
            if saida[i][j]>=255//2:
                saida[i][j]=255
            else:
                saida[i][j]=0
            er=salva-saida[i][j]
            if j+1<saida.shape[1]:
                saida[i][j+1]+=er/8
                if i+1<saida.shape[0]:
                    saida[i+1][j+1]+=er/8
            if i+1<saida.shape[0]:
                saida[i+1][j]+=er/8
                if j-1>=0:
                    saida[i+1][j-1]+=er/8
            if i+2<saida.shape[0]:
                saida[i+2][j]+=er/8
            if j+2<saida.shape[1]:
                saida[i][j+2]+=er/8
    imgAtkinson=Image.fromarray(np.uint8(saida))
    return imgAtkinson

#### Células de teste:

In [31]:
img = abre_imagem('blue_tang.jpg')
imgAtk = atkinson(img)
img.show()
imgAtk.show()

In [29]:
img = abre_imagem('capivara.jpg')
imgAtk = atkinson(img)
img.show()
imgAtk.show()

In [30]:
img = abre_imagem('praia.png')
imgAtk = atkinson(img)
img.show()
imgAtk.show()

### 2.2 Algoritmo de Bayer

<p>A ideia por trás deste algoritmo é aplicar um padrão de limiar sobre a imagem e binarizar a mesma de acordo com aquele padrão. Para isto, utiliza-se uma matriz de padrão, e subdivide-se a imagem em blocos do tamanho daquela matriz, de forma similar ao que fizemos com o efeito "pixelização". Se o valor do pixel for superior ao valor da posição correspondente na matriz de padrão, o pixel assume a cor branca. Caso contrário, assume a cor preta.</p>

<p>Temos duas versões clássicas de implementação do algoritmo. Com o padrão $2\times2$:</p>

<img src="bayer2.png">

<p>Dado pela matriz:</p>

\begin{equation}
    M = \begin{pmatrix}
    0 & 60 \\
    45 & 110 \\
    \end{pmatrix}\nonumber
\end{equation}

<p>E com o padrão $5\times5$:</p>

<img src="bayer5.png" width="15%">

<p>Dado pela matriz:</p>

\begin{equation}
    M = \begin{pmatrix}
    167 & 200 & 230 & 216 & 181 \\
    94 & 72 & 193 & 242 & 232 \\
    36 & 52 & 222 & 167 & 200 \\
    181 & 126 & 210 & 94 & 72\\
    232 & 153 & 111 & 36 & 52 \\
    \end{pmatrix}\nonumber
\end{equation}

<p>O algoritmo em alto nível para a técnica está a seguir.</p>

<img src="bayer.png" width="60%">

<p><font color="red">Implemente uma ou duas funções</font>, a seu critério, para aplicar o efeito sobre uma imagem de entrada. Implemente também os códigos das células de teste. Se for implementar uma única função, a matriz $M$ tem de ser passada como parâmetro.</p>

<p>Resultados esperados para a imagem <code>blue_tang.jpg</code> (esquerda: Bayer $2\times2$, direita: Bayer $5\times5$):</p>

<table>
    <tr>
        <td><img src="blue_tang_b2.png"></td>
        <td><img src="blue_tang_b5.png"></td>
    </tr>
</table>

<p>Resultado esperado para a imagem <code>capivara.jpg</code>  (esquerda: Bayer $2\times2$, direita: Bayer $5\times5$):</p>

<table>
    <tr>
        <td><img src="capivara_b2.jpg" width="60%"></td>
        <td><img src="capivara_b5.jpg" width="60%"></td>
    </tr>
</table>

<p>Resultado esperado para a imagem <code>praia.png</code>  (esquerda: Bayer $2\times2$, direita: Bayer $5\times5$):</p>

<table>
    <tr>
        <td><img src="praia_b2.png" width="60%"></td>
        <td><img src="praia_b5.png" width="60%"></td>
    </tr>
</table>

<p><b>Obs.:</b> note que, assim como no caso do algoritmo de Atkinson, a imagem de entrada precisa estar em tons de cinza.</p>

#### Implemente a(s) função(ões) na célula a seguir

In [38]:
def di(img,m):
    saida=np.asarray(img,dtype=int)
    d=m.shape[1]
    for i in range(saida.shape[0]):
        for j in range(saida.shape[1]):
            L=i%d
            C=j%d
            if saida[i][j]>m[L][C]:
                saida[i][j]=255
            else:
                saida[i][j]=0
    di=Image.fromarray(np.uint8(saida))
    return di

#### Código de teste para imagem <code>blue_tang.jpg</code>

In [39]:
img = abre_imagem('blue_tang.jpg')
M2 = np.array( [[0,60],[45,110]])
M5 = np.array(   [[167, 200, 230, 216, 181],
                [94, 72, 193, 242, 232],
                [36, 52, 222, 167, 200],
                [181, 126, 210, 94, 72],
                [232, 153, 111, 36, 52]])
img=ImageOps.grayscale(img)
imgdi2 = di(img,M2)
imgdi5 = di(img,M5)
img.show()
imgdi2.show()
imgdi5.show()

#### Código de teste para imagem <code>capivara.jpg</code>

In [40]:
img = abre_imagem('capivara.jpg')
M2 = np.array( [[0,60],[45,110]])
M5 = np.array(   [[167, 200, 230, 216, 181],
                [94, 72, 193, 242, 232],
                [36, 52, 222, 167, 200],
                [181, 126, 210, 94, 72],
                [232, 153, 111, 36, 52]])
img=ImageOps.grayscale(img)
imgdi2 = di(img,M2)
imgdi5 = di(img,M5)
img.show()
imgdi2.show()
imgdi5.show()

#### Código de teste para imagem <code>praia.png</code>

In [ ]:
img = abre_imagem('praia.png')
M2 = np.array( [[0,60],[45,110]])
M5 = np.array(   [[167, 200, 230, 216, 181],
                [94, 72, 193, 242, 232],
                [36, 52, 222, 167, 200],
                [181, 126, 210, 94, 72],
                [232, 153, 111, 36, 52]])
img=ImageOps.grayscale(img)
imgdi2 = di(img,M2)
imgdi5 = di(img,M5)
img.show()
imgdi2.show()
imgdi5.show()